This notebook's training loop has some minor issues with loss function and it has been forked 200+ times.<br/>
https://www.kaggle.com/maunish/clrp-pytorch-roberta-finetune

This notebooks solves those issues. Most of the changes are in run() function.<br/>
The important one is using rmse_score for scoring best model.<br/>
Happy coding and best of luck with the Competition.

In [ ]:
!pip install accelerate

In [ ]:
import os
import gc
import sys
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from accelerate import Accelerator
from transformers import (AutoModel,AutoTokenizer,AutoConfig,
                          AdamW,get_cosine_schedule_with_warmup)

from colorama import Fore, Back, Style
r_ = Fore.RED
b_ = Fore.BLUE
c_ = Fore.CYAN
g_ = Fore.GREEN
y_ = Fore.YELLOW
m_ = Fore.MAGENTA
sr_ = Style.RESET_ALL

In [ ]:
train_data = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_data = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
sample = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')

num_bins = int(np.floor(1 + np.log2(len(train_data))))
train_data.loc[:,'bins'] = pd.cut(train_data['target'],bins=num_bins,labels=False)

def rmse_score(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true,y_pred))

In [ ]:
config = {
    'lr': 2e-5,
    'wd':0.01,
    'num_workers':4,
    'batch_size':16,
    'valid_step':10,
    'max_len':256,
    'epochs':3,
    'nfolds':5,
    'seed':42,
    'model_path':'../input/clrp-roberta-base/clrp_roberta_base',
}

for i in range(config['nfolds']):
    os.makedirs(f'model{i}',exist_ok=True)

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONASSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(seed=config['seed'])

train_data['Fold'] = -1
kfold = StratifiedKFold(n_splits=config['nfolds'],shuffle=True,random_state=config['seed'])
for k , (train_idx,valid_idx) in enumerate(kfold.split(X=train_data,y=train_data['bins'])):
    train_data.loc[valid_idx,'Fold'] = k

In [ ]:
class CLRPDataset(Dataset):
    def __init__(self,df,tokenizer,max_len=128):
        self.excerpt = df['excerpt'].to_numpy()
        self.targets = df['target'].to_numpy()
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        
        target = torch.tensor(self.targets[idx],dtype=torch.float) 
        return encode, target
    
    def __len__(self):
        return len(self.excerpt)

In [ ]:
class AttentionHead(nn.Module):
    def __init__(self, in_features, hidden_dim):
        super().__init__()
        self.W = nn.Linear(in_features, hidden_dim)
        self.V = nn.Linear(hidden_dim, 1)

    def forward(self, features):
        att = torch.tanh(self.W(features))
        score = self.V(att)
        attention_weights = torch.softmax(score, dim=1)
        context_vector = torch.sum(attention_weights * features, dim=1)
        return context_vector

In [ ]:
class Model(nn.Module):
    def __init__(self,path):
        super(Model,self).__init__()
        self.config = AutoConfig.from_pretrained(path)        
        self.roberta = AutoModel.from_pretrained(path,config=self.config)  
        self.head = AttentionHead(self.config.hidden_size,self.config.hidden_size)
        self.linear = nn.Linear(self.config.hidden_size,1)

    def forward(self,**xb):
        x = self.roberta(**xb)[0]
        x = self.head(x)
        x = self.linear(x)
        return x

In [ ]:
def run(fold):
    
    def loss_fn(outputs,targets):
        return torch.sqrt(nn.MSELoss()(outputs.view(-1),targets.view(-1)))
    
    def evaluate(model,valid_loader):
        model.eval()
        valid_loss = 0
        all_targets, all_outputs = [], []
        with torch.no_grad():
            for i, (inputs,targets) in enumerate(valid_loader):
                inputs = {key:val.reshape(val.shape[0],-1) for key,val in inputs.items()}
                outputs = model(**inputs)
                valid_loss += loss_fn(outputs,targets).item()
                all_outputs.extend(outputs.cpu().detach().numpy().tolist())
                all_targets.extend(targets.cpu().detach().numpy().tolist())

        valid_loss /= len(valid_loader)
        valid_rmse = rmse_score(all_outputs,all_targets)
        return valid_loss,valid_rmse
        
    def train_and_evaluate_loop(train_loader,valid_loader,model,loss_fn,optimizer,
                                epoch,fold,best_score,valid_step,lr_scheduler=None):
        train_loss = 0
        for i, (inputs,targets) in enumerate(train_loader):
            optimizer.zero_grad()
            model.train()
            inputs = {key:val.reshape(val.shape[0],-1) for key,val in inputs.items()}
            outputs = model(**inputs)
            loss = loss_fn(outputs,targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            if lr_scheduler:
                lr_scheduler.step()
                        
            if (i% valid_step ==0) or ((i+1) == len(train_loader)):
                valid_loss,valid_rmse = evaluate(model,valid_loader) 
                        
                if valid_rmse <= best_score:
                    print(f"Epoch:{epoch}|Batch: {i}|Step:{valid_step}|Train Loss:{train_loss/(i+1)}|Valid Loss:{valid_loss}|Rmse Score: {valid_rmse}")
                    print(f"{g_}Rmse Score Decreased from {best_score} to {valid_rmse}{sr_}")

                    best_score = valid_rmse
                    torch.save(model.state_dict(),f'./model{fold}/model{fold}.bin')
                    tokenizer.save_pretrained(f'./model{fold}')
                    
        return best_score
        
    accelerator = Accelerator()
    print(f"{accelerator.device} is used")
    
    x_train,x_valid = train_data.query(f"Fold != {fold}"),train_data.query(f"Fold == {fold}")
    
    model = Model(config['model_path'])
    tokenizer = AutoTokenizer.from_pretrained(config['model_path'])
    
    train_ds = CLRPDataset(x_train,tokenizer,config['max_len'])
    train_dl = DataLoader(train_ds,
                        batch_size = config["batch_size"],
                        num_workers = config['num_workers'],
                        shuffle=True,
                        pin_memory=True,
                        drop_last=True)

    valid_ds = CLRPDataset(x_valid,tokenizer,config['max_len'])
    valid_dl = DataLoader(valid_ds,
                        batch_size = config["batch_size"],
                        num_workers = config['num_workers'],
                        shuffle=False,
                        pin_memory=True,
                        drop_last=False)
        
    optimizer = AdamW(model.parameters(),lr=config['lr'],weight_decay=config['wd'])    
    lr_scheduler = get_cosine_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps= config['epochs'] * len(train_dl))

    model,train_dl,valid_dl,optimizer,lr_scheduler = accelerator.prepare(model,train_dl,valid_dl,optimizer,lr_scheduler)

    print(f"Fold: {fold}")
    best_score = 9999
    start_time = time.time()
    for epoch in range(config["epochs"]):
        print(f"Epoch Started:{epoch}")
        best_score = train_and_evaluate_loop(train_dl,valid_dl,model,loss_fn,optimizer,epoch,fold,
                                             best_score,config['valid_step'],lr_scheduler)
        
        end_time = time.time()
        print(f"{m_}Time taken by epoch {epoch} is {end_time-start_time:.2f}s{sr_}")
        start_time = end_time
        
    return best_score

In [ ]:
best_score_per_fold = [run(f) for f in range(config['nfolds'])]

In [ ]:
print("Best Score Per Fold: ",best_score_per_fold)
print("Mean Score: ",np.mean(best_score_per_fold))